# L4: 过程输入： 思考链推理

## 配置
#### 加载 API 密钥和相关 Python 库。
在本课程中，我们提供了一些为您加载 OpenAI API 密钥的代码

In [28]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = 'API-KEY'

In [29]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

## 思维链提示

In [30]:
delimiter = "####"
system_message = f"""
按照这些步骤来回答客户的询问。 \
客户的查询将以四个标签为界、 即{delimiter}。

第1步：{delimiter}。首先决定用户是否在问一 \
个或多个具体产品的问题。产品名称不算数。

第2步：{delimiter}如果用户询问的是具体的产品，\
请识别该产品是否在以下列表中。
所有可用的产品：
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

第3步：{delimiter}如果信息中包含上述列表中的产品，请列出用户在信息中的任何假设，\
例如，笔记本电脑X比笔记本电脑Y大，或者笔记本电脑Z有2年的保修期。

第四步：{delimiter}： 如果用户做出了任何假设，\
请根据你的产品信息弄清楚该假设是否真实。

第五步：{delimiter}： 首先，礼貌地纠正顾客不正确的假设（如果适用）。\
只提及或参考5种可用产品清单中的产品，因为这是该店\
唯一出售的5种产品。用友好的语气回答顾客。

使用一下格式:
第一步:{delimiter} <步骤 1 推理>
第二步:{delimiter} <步骤 2 推理>
第三步:{delimiter} <步骤 3 推理>
第四步:{delimiter} <步骤 4 推理>
回复用户:{delimiter} <对客户的回应>

请确保在每个步骤之间使用{delimiter}作为分隔符。
"""

In [31]:
user_message = f"""
BlueWave Chromebook 比 TechPro Desktop贵多少？
"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

#### 步骤 1：确定用户是否在问一个或多个具体产品的问题。
这个问题是关于两个具体的产品：BlueWave Chromebook和TechPro Desktop。

#### 步骤 2：确定产品是否在产品列表中。
这两个产品都在产品列表中。

#### 步骤 3：列出用户在信息中的任何假设。
用户没有提到任何关于这两个产品的假设。

#### 步骤 4：确定假设是否真实。
没有假设需要验证。

#### 回复用户：
BlueWave Chromebook的价格是$249.99，而TechPro Desktop的价格是$999.99。因此，TechPro Desktop比BlueWave Chromebook贵$750。


In [32]:
user_message = f"""
你卖电视么"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
]
try:
    response = get_completion_from_messages(messages)
    print(response)
except Exception as e:
    print(e)

#### 步骤 1：推理
客户询问是否销售电视，这是一个具体的产品问题。

#### 步骤 2：推理
我们的产品清单中没有电视，因此我们不销售电视。

#### 步骤 3：推理
无需列出任何假设，因为客户没有提到任何特定的电视型号或品牌。

#### 步骤 4：推理
无需验证任何假设，因为客户没有提到任何特定的电视型号或品牌。

回复用户：#### 很抱歉，我们不销售电视。我们的产品清单中只有五种产品，分别是TechPro Ultrabook、BlueWave Gaming Laptop、PowerLite Convertible、TechPro Desktop和BlueWave Chromebook。


## 内心独白
- 由于我们要求语言模型使用分隔符分隔其推理步骤，我们可以将思维链的推理过程隐藏起来，不让用户看到最终输出。

In [33]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

很抱歉，我们不销售电视。我们的产品清单中只有五种产品，分别是TechPro Ultrabook、BlueWave Gaming Laptop、PowerLite Convertible、TechPro Desktop和BlueWave Chromebook。
